In [3]:
import pickle
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization, GlobalMaxPool2D, MaxPooling2D

In [4]:
image_size = 28
num_labels = 10

## Загрузка данных

In [5]:
with open('../data/lab_1/X_train.pickle', 'rb') as f:
    X_train = pickle.load(f)

with open('../data/lab_1/y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with open('../data/lab_1/X_test.pickle', 'rb') as f:
    X_test = pickle.load(f)

with open('../data/lab_1/y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)
    
with open('../data/lab_1/X_valid.pickle', 'rb') as f:
    X_valid = pickle.load(f)

with open('../data/lab_1/y_valid.pickle', 'rb') as f:
    y_valid = pickle.load(f)

In [6]:
def reformat(dataset, labels):
    dataset = dataset.reshape((dataset.shape[0], 28, 28, 1)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

In [7]:
X_train, y_train = reformat(X_train, y_train)
X_test, y_test = reformat(X_test, y_test)
X_valid, y_valid = reformat(X_valid, y_valid)

In [8]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                368650    
Total params: 406,218
Trainable params: 406,218
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(patience=10)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [10]:
model.fit(X_train, y_train,
          epochs=50,
          batch_size=512,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks,
          verbose=1)


Train on 300000 samples, validate on 93124 samples
Epoch 1/50
300000/300000 [==============================] - 186s 619us/step - loss: 0.8862 - accuracy: 0.7714 - val_loss: 0.7778 - val_accuracy: 0.7945
Epoch 2/50
300000/300000 [==============================] - 186s 620us/step - loss: 0.7108 - accuracy: 0.8152 - val_loss: 0.7239 - val_accuracy: 0.8092
Epoch 3/50
300000/300000 [==============================] - 188s 627us/step - loss: 0.6516 - accuracy: 0.8285 - val_loss: 0.6598 - val_accuracy: 0.8215
Epoch 4/50
300000/300000 [==============================] - 186s 619us/step - loss: 0.5890 - accuracy: 0.8421 - val_loss: 0.5957 - val_accuracy: 0.8369
Epoch 5/50
300000/300000 [==============================] - 187s 624us/step - loss: 0.5344 - accuracy: 0.8540 - val_loss: 0.5553 - val_accuracy: 0.8445
Epoch 6/50
300000/300000 [==============================] - 186s 619us/step - loss: 0.4973 - accuracy: 0.8627 - val_loss: 0.5238 - val_accuracy: 0.8531
Epoch 7/50
300000/300000 [==========

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hd16547/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-1b1917297289>", line 6, in <module>
    verbose=1)
  File "/home/hd16547/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "/home/hd16547/.local/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "/home/hd16547/.local/lib/python3.7/site-packages/tensorflow/python/keras/backend.py", line 3292, in __call__
    run_metadata=self.run_metadata)
  File "/home/hd16547/.local/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1458, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hd

KeyboardInterrupt: 

In [9]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                108170    
Total params: 108,810
Trainable params: 108,810
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=512,
          validation_data=(X_valid, y_valid),
          verbose=1)


Train on 300000 samples, validate on 93124 samples
Epoch 1/10
300000/300000 [==============================] - 44s 148us/step - loss: 0.9749 - accuracy: 0.7587 - val_loss: 0.7578 - val_accuracy: 0.7998
Epoch 2/10
300000/300000 [==============================] - 44s 147us/step - loss: 0.6953 - accuracy: 0.8188 - val_loss: 0.7231 - val_accuracy: 0.8082
Epoch 3/10
300000/300000 [==============================] - 47s 155us/step - loss: 0.6707 - accuracy: 0.8239 - val_loss: 0.7035 - val_accuracy: 0.8119
Epoch 4/10
300000/300000 [==============================] - 48s 160us/step - loss: 0.6546 - accuracy: 0.8274 - val_loss: 0.6898 - val_accuracy: 0.8141
Epoch 5/10
300000/300000 [==============================] - 46s 155us/step - loss: 0.6411 - accuracy: 0.8302 - val_loss: 0.6774 - val_accuracy: 0.8177
Epoch 6/10
300000/300000 [==============================] - 44s 147us/step - loss: 0.6278 - accuracy: 0.8331 - val_loss: 0.6649 - val_accuracy: 0.8212
Epoch 7/10
300000/300000 [================

In [4]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 48)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               307456    
_________________________________________________________________
dense_3 (Dense)              (None, 84)               

In [9]:
adam = Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [10]:
datagen = ImageDataGenerator(
            rotation_range=10, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            zoom_range=0.1)
datagen.fit(X_train)

In [11]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-6)

In [13]:
model.fit(datagen.flow(X_train, y_train, batch_size=512),
          epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks=[reduce_lr],
          verbose=1)

Epoch 1/10
586/586 [==============================] - 64s 109ms/step - loss: 0.3809 - accuracy: 0.8857 - val_loss: 0.3508 - val_accuracy: 0.8955
Epoch 2/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3746 - accuracy: 0.8879 - val_loss: 0.3446 - val_accuracy: 0.8971
Epoch 3/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3689 - accuracy: 0.8893 - val_loss: 0.3414 - val_accuracy: 0.8982
Epoch 4/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3629 - accuracy: 0.8906 - val_loss: 0.3387 - val_accuracy: 0.8987
Epoch 5/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3567 - accuracy: 0.8922 - val_loss: 0.3303 - val_accuracy: 0.9008
Epoch 6/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3519 - accuracy: 0.8942 - val_loss: 0.3297 - val_accuracy: 0.9009
Epoch 7/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3477 - accuracy: 0.8951 - val_loss: 0.3239 - val_ac